<center><a href="https://5loi.com/about_loi"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a></center>

# 文本分类 #
> Text Classification

### 联合意图和槽位分类 ##
> Joint Intent and Slot classification

意图和槽位分类是一项任务，旨在对查询进行意图分类，并检测与此意图相关的所有槽位（实体）。例如，在查询“明早圣克拉拉的天气如何？”中，我们希望将查询分类为“天气”意图，并检测“圣克拉拉”作为“位置”槽位，以及“明早”作为“日期时间”槽位。意图和槽位的名称通常特定于任务，并在训练数据中定义为标签。这是任何任务驱动的对话式助手执行的基本步骤。

NeMo 基于 BERT 的模型实现能够一起训练和检测这两项任务。


**目录**<br>
本笔记本涵盖以下部分：
* 数据集
    * 下载和预处理数据
    * 数据标记（可选）
* 模型训练
    * 下载模型
    * 进行预测
* 评估


In [ ]:
BRANCH = 'main'

In [ ]:
from nemo.collections import nlp as nemo_nlp
from nemo.utils import logging

import os
import wget
import torch
import pytorch_lightning as pl
from omegaconf import OmegaConf

## 数据集和 NeMo 数据格式 ##
> Dataset and NeMo data format

在本教程中，我们将使用一个虚拟助手交互数据集，可以从此处下载：https://github.com/xliuhw/NLU-Evaluation-Data。
大约有 10K 个训练查询和 1K 个测试查询，涵盖了 64 种不同的意图和 55 个槽位。

为了使用 NeMo NLP 分类模型，该数据集应首先转换为 NeMo 格式，这需要以下文件：

- **dict.intents.csv** - 数据中所有意图名称的列表。每个意图名称一行。
- **dict.slots.csv** - 数据中所有槽位名称的列表。每个槽位名称一行。可以同时使用 B- 和 I- 表示法，以区分多词槽位的第一个和中间词。或者，每个多词槽位的每个词都只使用一种槽位类型。我们建议使用后者，因为它更简单，并且性能没有明显的下降。
- **train.tsv/test.tsv** - 每一行包含一个原始查询，以及用制表符分隔的意图编号。例如：`what alarms do i have set right now	0`。意图编号根据意图字典文件（dict.intents.csv）中的意图行，从 0 开始。这些文件的首行包含标题行：`sentence \tab label`。
- **train_slot.tvs/test_slot.tsv** - 每一行包含一个查询，其中每个词都用槽位字典文件（dict.slots.csv）中的词编号替换，从 0 开始。最后一个“超出范围”的词通常位于字典的最后一行。示例：`54 0 0 54 54 12 12`（数字之间用空格分隔）。这些文件没有标题行。

我们提供了 `import_dataset.py` 和 `assistant_utils.py` 脚本，它们将数据转换为 NeMo 数据格式，用于意图和槽位分类模型。如果您自己的带注释的数据集格式不同，则需要编写数据转换器。对于您自己的注释，可能推荐的格式是为一个意图的所有示例创建一个文本文件。每个查询一行，格式如下：`did i set an alarm to [alarm_type : wake up] in the [timeofday : morning]`，使用方括号定义槽位名称。这与本示例中的助手格式非常相似，您可以使用其转换器进行少量修改，将其转换为 NeMo 格式。

您可以按如下方式运行此实用程序：
```
python examples/nlp/intent_slot_classification/data/import_datasets.py --dataset_name=assistant --source_data_dir=source_dir_name --target_data_dir=target_dir_name
```

#### 预处理数据 ####
> Preprocess Data

In [ ]:
# you can replace DATA_DIR and NEMO_DIR with your own locations
DATA_DIR = "data"
DATA_DIR=os.path.join(DATA_DIR, 'NLU-Evaluation-Data')

In [ ]:
# # download and unzip the example dataset from github
TEST_DIR='.'
print('Downloading dataset...')
wget.download('https://github.com/xliuhw/NLU-Evaluation-Data/archive/master.zip', TEST_DIR)
!unzip {TEST_DIR}/NLU-Evaluation-Data-master.zip -d {TEST_DIR}

In [ ]:
# convert the dataset to the NeMo format
!python import_datasets.py --dataset_name=assistant --source_data_dir={DATA_DIR} --target_data_dir={DATA_DIR}/nemo_format

## 数据探索
> Data exploration

您可以查看原始格式和 NeMo 格式的数据集。这里有 65 种不同的意图和 55 个槽位，这可能是虚拟助手的典型命令。超出范围的槽位名称为“O”，并且是槽位字典中的最后一个。我们还可以查看查询示例以及训练意图和槽位文件的格式。

In [ ]:
# list of queries divided by intent files in the original training dataset
! ls -l {DATA_DIR}/dataset/trainset

In [ ]:
# print all intents from the NeMo format intent dictionary
!echo 'Intents: ' $(wc -l < {DATA_DIR}/nemo_format/dict.intents.csv)
!cat {DATA_DIR}/nemo_format/dict.intents.csv

In [ ]:
# print all slots from the NeMo format slot dictionary
!echo 'Slots: ' $(wc -l < {DATA_DIR}/nemo_format/dict.slots.csv)
!cat {DATA_DIR}/nemo_format/dict.slots.csv

In [ ]:
# examples from the intent training file
!head -n 10 {DATA_DIR}/nemo_format/train.tsv

In [ ]:
# examples from the slot training file
!head -n 10 {DATA_DIR}/nemo_format/train_slots.tsv

# 训练模型
> Training model

## 模型配置
> Model configuration

我们的联合意图和槽位分类模型由预训练的 [BERT](https://arxiv.org/pdf/1810.04805.pdf) 模型组成，并在其之上添加了意图和槽位分类层。


所有模型和训练参数都在 **intent_slot_classification_config.yaml** 配置文件中定义。此文件位于文件夹 **examples/nlp/intent_slot_classification/conf/** 中。它包含两个主要部分：

- **model**: 所有与模型相关的参数 - 语言模型、标记分类器、优化器和调度器、数据集以及任何其他相关信息

- **trainer**: 传递给 PyTorch Lightning 的任何参数


出于教程的目的，我们将从存储库下载配置文件。如果您在本地安装了 NeMo 版本，则可以从上述文件夹中使用它。

In [ ]:
WORK_DIR='WORK_DIR'
MODEL_CONFIG = "intent_slot_classification_config.yaml"
config_dir = WORK_DIR + '/configs/'
os.makedirs(config_dir, exist_ok=True)
if not os.path.exists(config_dir + MODEL_CONFIG):
    print('Downloading config file...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/intent_slot_classification/conf/' + MODEL_CONFIG, config_dir)
else:
    print ('config file already exists')

In [ ]:
config_file = "intent_slot_classification_config.yaml"
config=OmegaConf.load(config_dir + "/" + config_file)

# print the entire configuration file
print(OmegaConf.to_yaml(config))

## 在配置中设置数据
> Setting up Data within the config

除其他外，配置文件包含名为 train_ds 和 validation_ds 的字典。这些是用于设置对应配置的数据集和数据加载器的配置。


转换实用程序在同一目录中创建训练和评估文件，因此我们需要将 `model.data_dir` 参数指定为此目录。另请注意，某些配置文件行（包括 `model.data_dir`）在路径位置使用 `???`，这意味着用户需要指定这些字段的值。


`config.model.intent_loss_weight` 参数 - 是意图和槽位损失之间训练损失的平衡，一个介于 0 到 1 之间的数字。其默认值为 0.6，这赋予意图损失略高的优先级，并且在经验上效果很好。如果您愿意，可以尝试修改此值。
您还可以尝试将 `config.model.class_balancing` 参数更改为 `weighted_loss`，并查看是否可以获得更高的准确率。


现在让我们将数据目录路径添加到配置中。


In [ ]:
config.model.data_dir = f'{DATA_DIR}/nemo_format'

## 构建 PyTorch Lightning 训练器
> Building the PyTorch Lightning Trainer

NeMo 模型主要是 PyTorch Lightning 模块 - 因此完全兼容 PyTorch Lightning 生态系统。`config.trainer.max_epochs` 参数定义了训练轮次(epochs)的次数。通常，50-100 个训练轮次或更少就足以在您的数据上进行训练。让我们实例化 Trainer 对象。


In [ ]:
# lets modify some trainer configs
# checks if we have GPU available and uses it
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
config.trainer.devices = 1
config.trainer.accelerator = accelerator

config.trainer.precision = 16 if torch.cuda.is_available() else 32

# for mixed precision training, uncomment the line below (precision should be set to 16 and amp_level to O1):
# config.trainer.amp_level = O1

# remove distributed training flags
config.trainer.strategy = 'auto'

# setup a small number of epochs for demonstration purposes of this tutorial
config.trainer.max_epochs = 5

trainer = pl.Trainer(**config.trainer)

## 初始化模型和训练
> Initializing the model and Training

数据集的初始统计信息将在训练开始时显示，然后在每个训练轮次之后将显示意图和槽位分类报告。

In [ ]:
# initialize the model
model = nemo_nlp.models.IntentSlotClassificationModel(config.model, trainer=trainer)

# train
trainer.fit(model)

经过 5 个训练轮次（这应该不超过几分钟），您可以预期此数据集的训练精度约为以下数值（对于此数据集，准确率将随着训练轮次的增加而逐渐提高，最高可达约 50 个训练轮次）：


```
意图：
    标签                                                精确率    召回率       F1 值          支持度   
    alarm_query (标签 ID: 0)                               94.74      94.74      94.74         19
    alarm_remove (标签 ID: 1)                             100.00     100.00     100.00         11
    alarm_set (标签 ID: 2)                                 85.71      94.74      90.00         19
    audio_volume_down (标签 ID: 3)                          0.00       0.00       0.00          8
    audio_volume_mute (标签 ID: 4)                        100.00      86.67      92.86         15
    audio_volume_up (标签 ID: 5)                           56.52     100.00      72.22         13
    calendar_query (标签 ID: 6)                            55.00      57.89      56.41         19
    calendar_remove (标签 ID: 7)                           88.89      84.21      86.49         19
    calendar_set (标签 ID: 8)                              81.25      68.42      74.29         19
    cooking_recipe (标签 ID: 9)                            86.36     100.00      92.68         19
    datetime_convert (标签 ID: 10)                          0.00       0.00       0.00          8
    datetime_query (标签 ID: 11)                           65.52     100.00      79.17         19
    email_addcontact (标签 ID: 12)                        100.00      12.50      22.22          8
    email_query (标签 ID: 13)                              83.33      78.95      81.08         19
    email_querycontact (标签 ID: 14)                       62.50      78.95      69.77         19
    email_sendemail (标签 ID: 15)                          70.83      89.47      79.07         19
    general_affirm (标签 ID: 16)                           95.00     100.00      97.44         19
    general_commandstop (标签 ID: 17)                     100.00     100.00     100.00         19
    general_confirm (标签 ID: 18)                         100.00     100.00     100.00         19
    general_dontcare (标签 ID: 19)                        100.00     100.00     100.00         19
    general_explain (标签 ID: 20)                         100.00      94.74      97.30         19
    general_joke (标签 ID: 21)                            100.00     100.00     100.00         12
    general_negate (标签 ID: 22)                           95.00     100.00      97.44         19
    general_praise (标签 ID: 23)                          100.00      94.74      97.30         19
    general_quirky (标签 ID: 24)                           40.00      10.53      16.67         19
    general_repeat (标签 ID: 25)                          100.00     100.00     100.00         19
    iot_cleaning (标签 ID: 26)                             84.21     100.00      91.43         16
    iot_coffee (标签 ID: 27)                               94.74      94.74      94.74         19
    iot_hue_lightchange (标签 ID: 28)                      94.44      89.47      91.89         19
    iot_hue_lightdim (标签 ID: 29)                        100.00      83.33      90.91         12
    iot_hue_lightoff (标签 ID: 30)                         89.47      89.47      89.47         19
    iot_hue_lighton (标签 ID: 31)                           0.00       0.00       0.00          3
    iot_hue_lightup (标签 ID: 32)                          81.25      92.86      86.67         14
    iot_wemo_off (标签 ID: 33)                             60.00     100.00      75.00          9
    iot_wemo_on (标签 ID: 34)                             100.00      14.29      25.00          7
    lists_createoradd (标签 ID: 35)                        78.95      78.95      78.95         19
    lists_query (标签 ID: 36)                              78.95      78.95      78.95         19
    lists_remove (标签 ID: 37)                             90.00      94.74      92.31         19
    music_likeness (标签 ID: 38)                           70.59      66.67      68.57         18
    music_query (标签 ID: 39)                              77.78      73.68      75.68         19
    music_settings (标签 ID: 40)                            0.00       0.00       0.00          7
    news_query (标签 ID: 41)                               77.78      73.68      75.68         19
    play_audiobook (标签 ID: 42)                           90.00      94.74      92.31         19
    play_game (标签 ID: 43)                                80.00      84.21      82.05         19
    play_music (标签 ID: 44)                               53.85      73.68      62.22         19
    play_podcasts (标签 ID: 45)                            89.47      89.47      89.47         19
    play_radio (标签 ID: 46)                               93.75      78.95      85.71         19
    qa_currency (标签 ID: 47)                              95.00     100.00      97.44         19
    qa_definition (标签 ID: 48)                            85.00      89.47      87.18         19
    qa_factoid (标签 ID: 49)                               45.16      73.68      56.00         19
    qa_maths (标签 ID: 50)                                100.00     100.00     100.00         14
    qa_stock (标签 ID: 51)                                 95.00     100.00      97.44         19
    recommendation_events (标签 ID: 52)                    94.44      89.47      91.89         19
    recommendation_locations (标签 ID: 53)                 94.74      94.74      94.74         19
    recommendation_movies (标签 ID: 54)                   100.00     100.00     100.00         10
    social_post (标签 ID: 55)                              90.00      94.74      92.31         19
    social_query (标签 ID: 56)                             94.74     100.00      97.30         18
    takeaway_order (标签 ID: 57)                           93.75      78.95      85.71         19
    takeaway_query (标签 ID: 58)                           85.71      94.74      90.00         19
    transport_query (标签 ID: 59)                          83.33      78.95      81.08         19
    transport_taxi (标签 ID: 60)                          100.00     100.00     100.00         18
    transport_ticket (标签 ID: 61)                         89.47      89.47      89.47         19
    transport_traffic (标签 ID: 62)                       100.00     100.00     100.00         19
    weather_query (标签 ID: 63)                           100.00      89.47      94.44         19
    -------------------
    微平均                                               85.04      85.04      85.04       1076
    宏平均                                               81.13      80.81      79.36       1076
    加权平均                                            84.10      85.04      83.54       1076


槽位：
    标签                                                精确率    召回率       F1 值          支持度   
    alarm_type (标签 ID: 0)                                 0.00       0.00       0.00          0
    app_name (标签 ID: 1)                                   0.00       0.00       0.00          6
    artist_name (标签 ID: 2)                                0.00       0.00       0.00         21
    audiobook_author (标签 ID: 3)                           0.00       0.00       0.00          1
    audiobook_name (标签 ID: 4)                             0.00       0.00       0.00         18
    business_name (标签 ID: 5)                             60.00      56.60      58.25         53
    business_type (标签 ID: 6)                              0.00       0.00       0.00         24
    change_amount (标签 ID: 7)                              0.00       0.00       0.00         25
    coffee_type (标签 ID: 8)                                0.00       0.00       0.00          4
    color_type (标签 ID: 9)                                 0.00       0.00       0.00         12
    cooking_type (标签 ID: 10)                              0.00       0.00       0.00          0
    currency_name (标签 ID: 11)                            84.09      75.51      79.57         49
    date (标签 ID: 12)                                     57.95      91.07      70.83        112
    definition_word (标签 ID: 13)                           0.00       0.00       0.00         20
    device_type (标签 ID: 14)                              74.55      51.25      60.74         80
    drink_type (标签 ID: 15)                                0.00       0.00       0.00          0
    email_address (标签 ID: 16)                             0.00       0.00       0.00         14
    email_folder (标签 ID: 17)                              0.00       0.00       0.00          1
    event_name (标签 ID: 18)                              100.00      13.24      23.38         68
    food_type (标签 ID: 19)                                51.72      69.77      59.41         43
    game_name (标签 ID: 20)                                60.00      14.29      23.08         21
    game_type (标签 ID: 21)                                 0.00       0.00       0.00          0
    general_frequency (标签 ID: 22)                         0.00       0.00       0.00          9
    house_place (标签 ID: 23)                              93.33      42.42      58.33         33
    ingredient (标签 ID: 24)                                0.00       0.00       0.00          6
    joke_type (标签 ID: 25)                                 0.00       0.00       0.00          4
    list_name (标签 ID: 26)                                 0.00       0.00       0.00         21
    meal_type (标签 ID: 27)                                 0.00       0.00       0.00          0
    media_type (标签 ID: 28)                                0.00       0.00       0.00         37
    movie_name (标签 ID: 29)                                0.00       0.00       0.00          0
    movie_type (标签 ID: 30)                                0.00       0.00       0.00          0
    music_album (标签 ID: 31)                               0.00       0.00       0.00          0
    music_descriptor (标签 ID: 32)                          0.00       0.00       0.00          3
    music_genre (标签 ID: 33)                               0.00       0.00       0.00          9
    news_topic (标签 ID: 34)                                0.00       0.00       0.00         17
    order_type (标签 ID: 35)                                0.00       0.00       0.00         17
    person (标签 ID: 36)                                   44.86      92.31      60.38         52
    personal_info (标签 ID: 37)                             0.00       0.00       0.00         20
    place_name (标签 ID: 38)                               71.25      77.03      74.03        148
    player_setting (标签 ID: 39)                            0.00       0.00       0.00          1
    playlist_name (标签 ID: 40)                             0.00       0.00       0.00          1
    podcast_descriptor (标签 ID: 41)                        0.00       0.00       0.00         13
    podcast_name (标签 ID: 42)                              0.00       0.00       0.00          4
    radio_name (标签 ID: 43)                               66.67      10.53      18.18         38
    relation (标签 ID: 44)                                  0.00       0.00       0.00         17
    song_name (标签 ID: 45)                                 0.00       0.00       0.00         22
    time (标签 ID: 46)                                     70.27      78.20      74.02        133
    time_zone (标签 ID: 47)                                 0.00       0.00       0.00          9
    timeofday (标签 ID: 48)                                 0.00       0.00       0.00         28
    transport_agency (标签 ID: 49)                          0.00       0.00       0.00          9
    transport_descriptor (标签 ID: 50)                      0.00       0.00       0.00          0
    transport_name (标签 ID: 51)                            0.00       0.00       0.00          4
    transport_type (标签 ID: 52)                           78.38      82.86      80.56         35
    weather_descriptor (标签 ID: 53)                        0.00       0.00       0.00         17
    O (标签 ID: 54)                                        92.42      98.80      95.50       5920
    -------------------
    微平均                                               89.10      89.10      89.10       7199
    宏平均                                               21.86      18.56      18.18       7199
    加权平均                                            84.42      89.10      86.01       7199
```


```
Intents:
    label                                                precision    recall       f1           support   
    alarm_query (label_id: 0)                               94.74      94.74      94.74         19
    alarm_remove (label_id: 1)                             100.00     100.00     100.00         11
    alarm_set (label_id: 2)                                 85.71      94.74      90.00         19
    audio_volume_down (label_id: 3)                          0.00       0.00       0.00          8
    audio_volume_mute (label_id: 4)                        100.00      86.67      92.86         15
    audio_volume_up (label_id: 5)                           56.52     100.00      72.22         13
    calendar_query (label_id: 6)                            55.00      57.89      56.41         19
    calendar_remove (label_id: 7)                           88.89      84.21      86.49         19
    calendar_set (label_id: 8)                              81.25      68.42      74.29         19
    cooking_recipe (label_id: 9)                            86.36     100.00      92.68         19
    datetime_convert (label_id: 10)                          0.00       0.00       0.00          8
    datetime_query (label_id: 11)                           65.52     100.00      79.17         19
    email_addcontact (label_id: 12)                        100.00      12.50      22.22          8
    email_query (label_id: 13)                              83.33      78.95      81.08         19
    email_querycontact (label_id: 14)                       62.50      78.95      69.77         19
    email_sendemail (label_id: 15)                          70.83      89.47      79.07         19
    general_affirm (label_id: 16)                           95.00     100.00      97.44         19
    general_commandstop (label_id: 17)                     100.00     100.00     100.00         19
    general_confirm (label_id: 18)                         100.00     100.00     100.00         19
    general_dontcare (label_id: 19)                        100.00     100.00     100.00         19
    general_explain (label_id: 20)                         100.00      94.74      97.30         19
    general_joke (label_id: 21)                            100.00     100.00     100.00         12
    general_negate (label_id: 22)                           95.00     100.00      97.44         19
    general_praise (label_id: 23)                          100.00      94.74      97.30         19
    general_quirky (label_id: 24)                           40.00      10.53      16.67         19
    general_repeat (label_id: 25)                          100.00     100.00     100.00         19
    iot_cleaning (label_id: 26)                             84.21     100.00      91.43         16
    iot_coffee (label_id: 27)                               94.74      94.74      94.74         19
    iot_hue_lightchange (label_id: 28)                      94.44      89.47      91.89         19
    iot_hue_lightdim (label_id: 29)                        100.00      83.33      90.91         12
    iot_hue_lightoff (label_id: 30)                         89.47      89.47      89.47         19
    iot_hue_lighton (label_id: 31)                           0.00       0.00       0.00          3
    iot_hue_lightup (label_id: 32)                          81.25      92.86      86.67         14
    iot_wemo_off (label_id: 33)                             60.00     100.00      75.00          9
    iot_wemo_on (label_id: 34)                             100.00      14.29      25.00          7
    lists_createoradd (label_id: 35)                        78.95      78.95      78.95         19
    lists_query (label_id: 36)                              78.95      78.95      78.95         19
    lists_remove (label_id: 37)                             90.00      94.74      92.31         19
    music_likeness (label_id: 38)                           70.59      66.67      68.57         18
    music_query (label_id: 39)                              77.78      73.68      75.68         19
    music_settings (label_id: 40)                            0.00       0.00       0.00          7
    news_query (label_id: 41)                               77.78      73.68      75.68         19
    play_audiobook (label_id: 42)                           90.00      94.74      92.31         19
    play_game (label_id: 43)                                80.00      84.21      82.05         19
    play_music (label_id: 44)                               53.85      73.68      62.22         19
    play_podcasts (label_id: 45)                            89.47      89.47      89.47         19
    play_radio (label_id: 46)                               93.75      78.95      85.71         19
    qa_currency (label_id: 47)                              95.00     100.00      97.44         19
    qa_definition (label_id: 48)                            85.00      89.47      87.18         19
    qa_factoid (label_id: 49)                               45.16      73.68      56.00         19
    qa_maths (label_id: 50)                                100.00     100.00     100.00         14
    qa_stock (label_id: 51)                                 95.00     100.00      97.44         19
    recommendation_events (label_id: 52)                    94.44      89.47      91.89         19
    recommendation_locations (label_id: 53)                 94.74      94.74      94.74         19
    recommendation_movies (label_id: 54)                   100.00     100.00     100.00         10
    social_post (label_id: 55)                              90.00      94.74      92.31         19
    social_query (label_id: 56)                             94.74     100.00      97.30         18
    takeaway_order (label_id: 57)                           93.75      78.95      85.71         19
    takeaway_query (label_id: 58)                           85.71      94.74      90.00         19
    transport_query (label_id: 59)                          83.33      78.95      81.08         19
    transport_taxi (label_id: 60)                          100.00     100.00     100.00         18
    transport_ticket (label_id: 61)                         89.47      89.47      89.47         19
    transport_traffic (label_id: 62)                       100.00     100.00     100.00         19
    weather_query (label_id: 63)                           100.00      89.47      94.44         19
    -------------------
    micro avg                                               85.04      85.04      85.04       1076
    macro avg                                               81.13      80.81      79.36       1076
    weighted avg                                            84.10      85.04      83.54       1076
    
Slots:
    label                                                precision    recall       f1           support   
    alarm_type (label_id: 0)                                 0.00       0.00       0.00          0
    app_name (label_id: 1)                                   0.00       0.00       0.00          6
    artist_name (label_id: 2)                                0.00       0.00       0.00         21
    audiobook_author (label_id: 3)                           0.00       0.00       0.00          1
    audiobook_name (label_id: 4)                             0.00       0.00       0.00         18
    business_name (label_id: 5)                             60.00      56.60      58.25         53
    business_type (label_id: 6)                              0.00       0.00       0.00         24
    change_amount (label_id: 7)                              0.00       0.00       0.00         25
    coffee_type (label_id: 8)                                0.00       0.00       0.00          4
    color_type (label_id: 9)                                 0.00       0.00       0.00         12
    cooking_type (label_id: 10)                              0.00       0.00       0.00          0
    currency_name (label_id: 11)                            84.09      75.51      79.57         49
    date (label_id: 12)                                     57.95      91.07      70.83        112
    definition_word (label_id: 13)                           0.00       0.00       0.00         20
    device_type (label_id: 14)                              74.55      51.25      60.74         80
    drink_type (label_id: 15)                                0.00       0.00       0.00          0
    email_address (label_id: 16)                             0.00       0.00       0.00         14
    email_folder (label_id: 17)                              0.00       0.00       0.00          1
    event_name (label_id: 18)                              100.00      13.24      23.38         68
    food_type (label_id: 19)                                51.72      69.77      59.41         43
    game_name (label_id: 20)                                60.00      14.29      23.08         21
    game_type (label_id: 21)                                 0.00       0.00       0.00          0
    general_frequency (label_id: 22)                         0.00       0.00       0.00          9
    house_place (label_id: 23)                              93.33      42.42      58.33         33
    ingredient (label_id: 24)                                0.00       0.00       0.00          6
    joke_type (label_id: 25)                                 0.00       0.00       0.00          4
    list_name (label_id: 26)                                 0.00       0.00       0.00         21
    meal_type (label_id: 27)                                 0.00       0.00       0.00          0
    media_type (label_id: 28)                                0.00       0.00       0.00         37
    movie_name (label_id: 29)                                0.00       0.00       0.00          0
    movie_type (label_id: 30)                                0.00       0.00       0.00          0
    music_album (label_id: 31)                               0.00       0.00       0.00          0
    music_descriptor (label_id: 32)                          0.00       0.00       0.00          3
    music_genre (label_id: 33)                               0.00       0.00       0.00          9
    news_topic (label_id: 34)                                0.00       0.00       0.00         17
    order_type (label_id: 35)                                0.00       0.00       0.00         17
    person (label_id: 36)                                   44.86      92.31      60.38         52
    personal_info (label_id: 37)                             0.00       0.00       0.00         20
    place_name (label_id: 38)                               71.25      77.03      74.03        148
    player_setting (label_id: 39)                            0.00       0.00       0.00          1
    playlist_name (label_id: 40)                             0.00       0.00       0.00          1
    podcast_descriptor (label_id: 41)                        0.00       0.00       0.00         13
    podcast_name (label_id: 42)                              0.00       0.00       0.00          4
    radio_name (label_id: 43)                               66.67      10.53      18.18         38
    relation (label_id: 44)                                  0.00       0.00       0.00         17
    song_name (label_id: 45)                                 0.00       0.00       0.00         22
    time (label_id: 46)                                     70.27      78.20      74.02        133
    time_zone (label_id: 47)                                 0.00       0.00       0.00          9
    timeofday (label_id: 48)                                 0.00       0.00       0.00         28
    transport_agency (label_id: 49)                          0.00       0.00       0.00          9
    transport_descriptor (label_id: 50)                      0.00       0.00       0.00          0
    transport_name (label_id: 51)                            0.00       0.00       0.00          4
    transport_type (label_id: 52)                           78.38      82.86      80.56         35
    weather_descriptor (label_id: 53)                        0.00       0.00       0.00         17
    O (label_id: 54)                                        92.42      98.80      95.50       5920
    -------------------
    micro avg                                               89.10      89.10      89.10       7199
    macro avg                                               21.86      18.56      18.18       7199
    weighted avg                                            84.42      89.10      86.01       7199
```

## 评估
> Evaluation

In [ ]:
# we will setup testing data reusing the same config (test section)
model.setup_test_data(test_data_config=config.model.test_ds)

# run the evaluation on the test dataset
trainer.test(model=model, verbose=False)

## 从示例进行推理
> Inference from Examples

下一步是查看训练后的模型将如何对来自此领域的给定查询进行意图和槽位分类。为了改进预测，您可能需要训练模型超过 5 个训练轮次。

In [ ]:
queries = [
    'set alarm for seven thirty am',
    'lower volume by fifty percent',
    'what is my schedule for tomorrow',
]

# pred_intents, pred_slots = eval_model.predict_from_examples(queries, config.model.test_ds)
pred_intents, pred_slots = model.predict_from_examples(queries, config.model.test_ds)

logging.info('The prediction results of some sample queries with the trained model:')
for query, intent, slots in zip(queries, pred_intents, pred_slots):
    logging.info(f'Query : {query}')
    logging.info(f'Predicted Intent: {intent}')
    logging.info(f'Predicted Slots: {slots}')

<center><a href="https://5loi.com/about_loi"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a></center>